# Modèle Prédictif par Segmentation (DPE vs Réel)

**Objectif** : Modéliser la relation entre la consommation théorique (DPE) et la consommation réelle en utilisant une approche par segmentation.

## 1. Importation des librairies et chargement des données

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configuration de l'affichage
pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")

In [ ]:
# Chargement du fichier fusionné (ECHANTILLON pour test rapide)
path_data = r'c:\Users\elean\OneDrive\Bureau\DPE\fusion_dpe_conso.csv'

# On charge seulement 100 000 lignes pour tester le modèle rapidement
# Pour charger tout le fichier, enlevez l'argument 'nrows=100000' ci-dessous
df = pd.read_csv(path_data, sep=',', low_memory=False, nrows=100000)

print(f"Dimensions du dataset (Echantillon) : {df.shape}")
df.head(3)

## 2. Préparation des données (Data Cleaning)

Nous allons :
1. Identifier la variable cible (Consommation Réelle) et la feature principale (DPE).
2. Harmoniser les unités (MWh vers kWh).
3. Filtrer les valeurs aberrantes et manquantes.

In [ ]:
# Noms de colonnes identifiées
col_real_mwh = "Consommation annuelle moyenne par logement de l'adresse (MWh)"
col_dpe_ef_kwh = "conso_5 usages_ef"  # Consommation Finale DPE (5 usages) en kWh

# Vérification de l'existence des colonnes
if col_real_mwh not in df.columns or col_dpe_ef_kwh not in df.columns:
    print("ATTENTION : Noms de colonnes incorrects. Vérifiez le header.")
    print(df.columns.tolist())
else:
    # Conversion et Renommage
    df['Conso_Reelle_kWh'] = df[col_real_mwh] * 1000  # MWh -> kWh
    df['Conso_DPE_kWh'] = df[col_dpe_ef_kwh]
    
    # Nettoyage des NaNs
    initial_len = len(df)
    df_clean = df.dropna(subset=['Conso_Reelle_kWh', 'Conso_DPE_kWh', 'type_batiment'])
    
    # Filtrage des valeurs absurdes (ex: conso négative ou nulle)
    df_clean = df_clean[(df_clean['Conso_Reelle_kWh'] > 0) & (df_clean['Conso_DPE_kWh'] > 0)]
    
    # Optionnel : Filtrer les outliers extrêmes (ex: > 100 000 kWh pour un logement)
    # df_clean = df_clean[df_clean['Conso_Reelle_kWh'] < 100000]
    
    print(f"Lignes conservées : {len(df_clean)} / {initial_len} ({len(df_clean)/initial_len:.1%})")
    
    # Aperçu
    df_clean[['type_batiment', 'Conso_DPE_kWh', 'Conso_Reelle_kWh']].head()

## 3. Analyse Exploratoire & Segmentation
Visualisons la relation DPE vs Réel globalement, puis par type de bâtiment.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean.sample(1000, random_state=42), x='Conso_DPE_kWh', y='Conso_Reelle_kWh', hue='type_batiment', alpha=0.5)
plt.plot([0, 50000], [0, 50000], 'r--', label='Identité (x=y)')
plt.title("Consommation Réelle vs DPE (Échantillon 1000 pts)")
plt.xlim(0, 50000)
plt.ylim(0, 50000)
plt.legend()
plt.show()

## 4. Modélisation : Approche Globale vs Segmentée
Nous allons comparer :
1. **Modèle Global** : 1 seule régression linéaire pour tous.
2. **Modèles Segmentés** : 1 régression par type de bâtiment (Maison, Appartement, etc.).

In [ ]:
results = []

def train_evaluate(data, name):
    if len(data) < 10:
        return None
        
    X = data[['Conso_DPE_kWh']]
    y = data['Conso_Reelle_kWh']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return {
        'Segment': name,
        'Count': len(data),
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Coef_Director': model.coef_[0],
        'Intercept': model.intercept_
    }

# 1. Modèle Global
res_global = train_evaluate(df_clean, "Global")
results.append(res_global)

# 2. Segmentation par Type de Bâtiment
segments = df_clean['type_batiment'].unique()
for seg in segments:
    data_seg = df_clean[df_clean['type_batiment'] == seg]
    res_seg = train_evaluate(data_seg, f"Type: {seg}")
    if res_seg:
        results.append(res_seg)

# Affichage des résultats
df_results = pd.DataFrame(results)
df_results

## 5. Analyse des Résultats

Le tableau ci-dessus permet de comparer la performance (R², RMSE) des modèles.
- Si le **R²** est plus élevé dans les segments que dans le global, la segmentation est pertinente.
- Le **Coefficient Directeur** indique la tendance : si < 1, le DPE surestime la consommation réelle (phénomène fréquent de précarité énergétique ou sobriété). Si > 1, le DPE sous-estime.

L'étape suivante consisterait à tester d'autres segmentations (ex: 'periode_construction') ou des modèles non-linéaires.